#### How to use:
- Setup your conda environment and install packages (Ask Dirk for help if needed)
- Download your lif file and put it in the 'test-files' folder
- Run the first 3 cells and select the file of interest
- Run the 4th cell and select which images you want to get the maximum projection images from
- Run the last cell and find your images in the 'maximum-projections' folder.

In [1]:
# load packages
from PIL import Image, ImageDraw, ImageFont
from readlif.reader import LifFile
import numpy as np
import napari
import ipywidgets as widgets
import os
from skimage import filters, util, draw
import cv2

In [45]:
# load functions
def collect_images(raw_data, img_sel):
    image_dict_list = []

    # check how many stacks, channels and mosaics there are
    raw_image = raw_data.get_image(img_sel)
    image_name = raw_image.name
    scale = tuple(abs(1/x) for x in raw_image.scale if x != None)
    z_nr_list = list(range(0,len([i for i in raw_image.get_iter_z()])))
    c_nr_list = list(range(0,len([i for i in raw_image.get_iter_c()])))
    m_nr_list = list(range(0,len([i for i in raw_image.get_iter_m()])))
    DimX = raw_image.dims[0]
    DimY = raw_image.dims[1]
    DimZ = raw_image.dims[2]

    # get info on if it is a mosaic tile
    M_positions = raw_image.mosaic_position # list with lenght of nr mosaics, each tuple contains (FieldX, FieldY, PosX, PosY)
    mosaic_list = list(range(0,len(M_positions)))
    mosaic_indication = 1
    if mosaic_list == []:
        mosaic_list = [0]
        mosaic_indication = 0 

    for m in m_nr_list:
        # collect info
        image_dict = {}
        image_dict['image_name'] = image_name
        image_dict['i_nr'] = img_sel
        image_dict['m_nr'] = m
        image_dict['scale'] = scale
        image_dict['M_positions'] = M_positions
        image_dict['DimX'] = DimX
        image_dict['DimY'] = DimY
        image_dict['DimZ'] = DimZ
        image_dict['bitdepth'] = raw_image.info['bit_depth'][0]
        # create 2d numpy array's per channel
        channel_dict = {}
        for c_nr in c_nr_list:
            for z_nr in z_nr_list:
                if z_nr == 0:
                    layers = []
                
                layer = np.asarray(raw_image.get_frame(z = z_nr, t = 0, c = c_nr, m = m))
                layers.append(layer)
            layers = np.stack(layers, axis = 2)  
            image_dict[f'channel_{c_nr}_image'] = layers
        
        image_dict_list.append(image_dict)
    return image_dict_list

def get_rgb(color):
    if color == 'blue':
        rgb = (0,0,1)
    elif color == 'green':
        rgb = (0,1,0)
    elif color == 'red':
        rgb = (1,0,0)
    elif color == 'yellow':
        rgb = (1,1,0)
    elif color == 'cyan':
        rgb = (0,1,1)
    elif color == 'magenta':
        rgb = (1,0,1)
    return rgb

def convert_intensity_range(img, target_type_min, target_type_max, target_type, min_quantile = False, max_quantile = False, input_bits = False):
    imin = img.min()
    # if no input bits is given, we want to transform the intensity range, not the type
    if input_bits == False:
        # correct if the image is binary and find min value
        if imin == False:
            img = img*1
            imin = img.min()
        if min_quantile != False:
            imin = np.quantile(img, q = min_quantile)
            
        # find max value
        imax = img.max()
        if max_quantile != False:
            imax = np.quantile(img, q = max_quantile)
    # if the input bits is given, we want to change the bitdepth, so we want to use the entire range
    else:
        if input_bits == 8:
            imin = 0
            imax = 255
        elif input_bits == 16:
            imin = 0
            imax = 65535
        elif input_bits == 32:
            imin = 0
            imax = 2147483647
    
    # build new image
    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b)
    new_img[new_img>255] = 255
    new_img[new_img<0] = 0
    new_img = new_img.astype(target_type)
    return new_img

def findHomography(image_1_kp, image_2_kp, matches):
    image_1_points = np.zeros((len(matches), 1, 2), dtype=np.float32)
    image_2_points = np.zeros((len(matches), 1, 2), dtype=np.float32)

    for i in range(0,len(matches)):
        image_1_points[i] = image_1_kp[matches[i].queryIdx].pt
        image_2_points[i] = image_2_kp[matches[i].trainIdx].pt


    homography, mask = cv2.findHomography(image_1_points, image_2_points, cv2.RANSAC, ransacReprojThreshold=2.0)

    return homography

def doLap(image):

    # YOU SHOULD TUNE THESE VALUES TO SUIT YOUR NEEDS
    kernel_size = 9         # Size of the laplacian window
    blur_size = 9          # How big of a kernal to use for the gaussian blur
                            # Generally, keeping these two values the same or very close works well
                            # Also, odd numbers, please...

    blurred = cv2.GaussianBlur(image, (blur_size,blur_size), 0)
    return cv2.Laplacian(blurred, cv2.CV_64F, ksize=kernel_size)

def focus_stack(images):
    """
    inspiration from here:
    https://github.com/cmcguinness/focusstack/blob/master/main.py
    """

    laps = []
    for i in range(len(images)):
        laps.append(doLap(cv2.cvtColor(images[i],cv2.COLOR_BGR2GRAY)))

    laps = np.asarray(laps)
    output = np.zeros(shape=images[0].shape, dtype=images[0].dtype)

    abs_laps = np.absolute(laps)
    maxima = abs_laps.max(axis=0)
    bool_mask = abs_laps == maxima
    mask = bool_mask.astype(np.uint8)
    for i in range(0,len(images)):
        output = cv2.bitwise_not(images[i],output, mask=mask[i])
		
    return 255-output



In [31]:
# specify file
location = 'test-files'
file_list = [f for f in os.listdir(location) if os.path.isfile(os.path.join(location, f))]

file_selection = widgets.Dropdown(options = file_list, description = 'Image name')
display(file_selection)

Dropdown(description='Image name', options=('SA_22.083_08-07-2022-new.lif', '22.088_22.103_TH_20220816.lif', '…

In [48]:
# specify image and scale bar
raw_data = LifFile(f"{location}/{file_selection.value}")
image_list = list(range(0,len([i for i in raw_data.get_iter_image()])))
image_name_list = []
for i in image_list:
    raw_image = raw_data.get_image(i)
    image_name_list.append(raw_image.name)

image_selection = widgets.Dropdown(options = ['All'] + image_name_list, description = 'Image name')
display(image_selection)

scale_bar_selection = widgets.Dropdown(options = [100, 250, 500, 750, 1000], description = 'Scale bar size')
display(scale_bar_selection)

Dropdown(description='Image name', options=('All', '03%_1', '03%_2', '03%_3', '05%_1', '05%_2', '05%_3'), valu…

Dropdown(description='Scale bar size', options=(100, 250, 500, 750, 1000), value=100)

In [49]:
# make the images
#  select the right order of colors in the image: (channel 0 has color 0, channel 1 has color 1 etc...)
color_list = ['blue', 'green', 'red', 'yellow', 'magenta', 'cyan', 'gray']

# select the folder in which you want to save your images
focus_projection_folder = 'focus_projection_images'

# load the data and get a list of all image numbers
raw_data = LifFile(f"{location}/{file_selection.value}")

if image_selection.value == 'All':
    image_list = list(range(0,len([i for i in raw_data.get_iter_image()])))
else:
    image_list = [n for n, x in enumerate(image_name_list) if x == image_selection.value]

# make the focus projection images per image
for i in image_list:
    # get the raw image
    raw_image = raw_data.get_image(i)
    # image_name = raw_image.name.split('/')[-1]
    image_name = raw_image.name.replace('/', '-')
    image_dict_list = collect_images(raw_data, i)
    for image_dict in image_dict_list:
        focus_dict = {}
        for n, (name, channel_image) in enumerate([(name, img) for name, img in image_dict.items() if 'channel' in name ]):
            channel_image = convert_intensity_range(channel_image, 0, 255, input_bits=image_dict['bitdepth'], target_type=np.uint8)
            # make channel stack
            channel_stack = []
            for z in range(channel_image.shape[2]):
                channel_stack.append(cv2.cvtColor(channel_image[:,:,z], cv2.COLOR_GRAY2BGR))
            focus_channel_image = convert_intensity_range(np.max(focus_stack(channel_stack), axis = 2), 0, 255, input_bits=image_dict['bitdepth'], target_type=np.uint8)

            # save the focus projection image of this channel, and assign a color
            focus_dict[f'{name}_focus_projection'] = {"image" : focus_channel_image, 'color' : color_list[n]}
        
        # get the m_nr
        m_nr = image_dict['m_nr']

        # build focus projection image
        empty_image = np.zeros_like(focus_dict['channel_0_image_focus_projection']['image'])
        default_color_image = np.dstack((empty_image, empty_image, empty_image))
        red_image = default_color_image[:,:,0]
        green_image = default_color_image[:,:,1]
        blue_image = default_color_image[:,:,2]

        # build the masked image, start with the default image
        for image_channel in focus_dict.keys():
            image, color = focus_dict[image_channel].values()
            print
            # change the color into rgb signals
            rgb = get_rgb(color)
            # fill in the image in the right color channels
            if rgb[0] > 0:
                red_image = np.dstack((red_image, image))
            if rgb[1] > 0:
                green_image = np.dstack((green_image, image))
            if rgb[2] > 0:
                blue_image = np.dstack((blue_image, image))
        
        # use maximum value if there are multiple stacks
        if len(red_image.shape)>2:
            red_image = np.max(red_image, axis = 2)
        if len(green_image.shape)>2:
            green_image = np.max(green_image, axis = 2)
        if len(blue_image.shape)>2:
            blue_image = np.max(blue_image, axis = 2)
        
        # design scalebar
        scale_x = image_dict['scale'][0]
        scale_bar_length = scale_bar_selection.value
        pxls_for_scale_bar_x = int(scale_bar_length/scale_x)
        pxls_for_scale_bar_y = 5
        start = ((image.shape[0]-pxls_for_scale_bar_x-30), (image.shape[1]-pxls_for_scale_bar_y-30))
        extent = (pxls_for_scale_bar_x, pxls_for_scale_bar_y)
        rr, cc = draw.rectangle(start, extent=extent, shape = image.shape)

        # insert rectangle
        red_image[cc,rr] = 255
        green_image[cc,rr] = 255
        blue_image[cc,rr] = 255
        
        # create the color image with dimensions (x, y, (r, g, b))
        color_image = np.dstack((red_image, green_image, blue_image))
        image_for_saving = Image.fromarray(convert_intensity_range(color_image, 0, 255, np.uint8, input_bits=8))   

        # design text
        scale_text = f'{scale_bar_length} um'
        font = ImageFont.truetype('DejaVuSansMono.ttf', 13, encoding='unic', )

        # draw text on the scale bar
        pil_draw = ImageDraw.Draw(image_for_saving)
        pil_draw.text((start[0]+int((pxls_for_scale_bar_x*0.35)),start[1]-20), scale_text, (255, 255, 255), font=font)

        #save the image with the correct name in the correct location
        image_for_saving.save(f"{focus_projection_folder}/{image_name}_{m_nr}.png")
